### Loading Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

### Loading Data

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
store_df = pd.read_csv("store.csv")

/Users/pratheerthpadman/anaconda/envs/carnd-term1/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [4]:
test_df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [5]:
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


### Dataset Stats

In [6]:
train_df.info()
print("-" * 30)
test_df.info()
print("-" * 30)
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
Store            1017209 non-null int64
DayOfWeek        1017209 non-null int64
Date             1017209 non-null object
Sales            1017209 non-null int64
Customers        1017209 non-null int64
Open             1017209 non-null int64
Promo            1017209 non-null int64
StateHoliday     1017209 non-null object
SchoolHoliday    1017209 non-null int64
dtypes: int64(7), object(2)
memory usage: 69.8+ MB
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
Id               41088 non-null int64
Store            41088 non-null int64
DayOfWeek        41088 non-null int64
Date             41088 non-null object
Open             41077 non-null float64
Promo            41088 non-null int64
StateHoliday     41088 non-null object
SchoolHoliday    41088 non-null int64
dtypes: float64(1), int64(5)

##### Training set contains no null values. There are some null values in the test set "Open" feature. Multiple features with null values in the store data

In [7]:
print ("The training dataset has {} samples and {} features".format(*train_df.shape))
print ("The testing dataset has {} samples and {} features".format(*test_df.shape))
print ("The store dataset has {} sample and {} features".format(*store_df.shape))

The training dataset has 1017209 samples and 9 features
The testing dataset has 41088 samples and 8 features
The store dataset has 1115 sample and 10 features


### Cleaning dataset

In [8]:
# Column - Date
print("Fixing the Timestamps")
train_df["Date"] = pd.to_datetime(train_df["Date"], format="%Y-%m-%d")
test_df["Date"] = pd.to_datetime(train_df["Date"], format="%Y-%m-%d")

Fixing the Timestamps


In [9]:
# Column - Open
print ("Assume store is open if nothing specified")
test_df["Open"].fillna(1, inplace=True)

Assume store is open if nothing specified


In [10]:
# Column - State Holiday
train_df["StateHoliday"].value_counts()

0    855087
0    131072
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64

In [11]:
test_df["StateHoliday"].value_counts()

0    40908
a      180
Name: StateHoliday, dtype: int64

In [12]:
#In the training dataset, StateHoliday has values 0 and "0". We'll need to merge those
train_df["StateHoliday"].loc[train_df["StateHoliday"]==0] = "0"

/Users/pratheerthpadman/anaconda/envs/carnd-term1/lib/python3.5/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
# StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. 
#Note that all schools are closed on public holidays and weekends.
#a = public holiday, b = Easter holiday, c = Christmas, 0 = None

In [14]:
#Now the a, b, c categories need to mapped to Holidays
train_df["StateHoliday"] = train_df["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})
test_df["StateHoliday"] = test_df["StateHoliday"].map({0: 0, "0": 0, "a": 1, "b": 1, "c": 1})

In [15]:
#Store Dataset
#Column - Competition Distance
store_df["CompetitionDistance"].fillna(store_df["CompetitionDistance"].mean(), inplace=True)

In [16]:
#Column - DayofWeek
train_df = pd.concat([train_df, pd.get_dummies(train_df["DayOfWeek"], prefix="DOW_")], axis=1)
test_df = pd.concat([test_df, pd.get_dummies(test_df["DayOfWeek"], prefix="DOW_")], axis=1)

train_df.drop("DayOfWeek", axis=1, inplace=True)
test_df.drop("DayOfWeek", axis=1, inplace=True)

In [17]:
#Column - StoreType
#store_df = pd.concat([store_df, pd.get_dummies(store_df["StoreType"], prefix="StoreType_")], axis=1)
store_df["StoreType"] = store_df["StoreType"].map({"a": 1, "b": 2, "c": 3, "d": 4})

In [18]:
#Column - Assortment
#Assortment - describes an assortment level: a = basic, b = extra, c = extended
store_df["Assortment"] = store_df["Assortment"].map({"a": 1, "b": 2, "c": 3})

### Feature Engineering

In [19]:
train_df["Month"] = train_df["Date"].dt.month
train_df["Year"] = train_df["Date"].dt.year
train_df["Day"] = train_df["Date"].dt.day
train_df["DayofYear"] = train_df["Date"].dt.dayofyear
train_df["WeekofYear"] = train_df["Date"].dt.weekofyear

In [20]:
test_df["Month"] = test_df["Date"].dt.month
test_df["Year"] = test_df["Date"].dt.year
test_df["Day"] = test_df["Date"].dt.day
test_df["DayofYear"] = test_df["Date"].dt.dayofyear
test_df["WeekofYear"] = test_df["Date"].dt.weekofyear

In [21]:
# Avg sales per store
sale_mean_per_store = train_df.groupby("Store").Sales.mean()
sale_mean_per_store.name = "Mean Sales Per Store"

In [22]:
# Avg customers per store
cust_mean_per_store = train_df.groupby("Store").Customers.mean()
cust_mean_per_store.name = "Mean Customers Per Store"

In [23]:
#sale_mean_per_day = train_df.groupby("DayofYear").Sales.mean()
#sale_mean_per_day.name = "Mean Sales Per Day"

In [24]:
#cust_mean_per_day = train_df.groupby("DayofYear").Customers.mean()
#cust_mean_per_day.name = "Mean Customers Per Day"

In [25]:
train_df = train_df.join(sale_mean_per_store, on="Store")
test_df = test_df.join(sale_mean_per_store, on="Store")

train_df = train_df.join(cust_mean_per_store, on="Store")
test_df = test_df.join(cust_mean_per_store, on="Store")

#train_df = train_df.join(sale_mean_per_day, on="WeekofYear")
#test_df = test_df.join(sale_mean_per_day, on="WeekofYear")

#train_df = train_df.join(cust_mean_per_day, on="WeekofYear")
#test_df = test_df.join(cust_mean_per_day, on="WeekofYear")

In [26]:
# Dropping columns customers since it is absent in the testing set
train_df.drop("Customers", axis=1, inplace=True)

In [27]:
train_df.drop("Date", axis=1, inplace=True)
test_df.drop("Date", axis=1, inplace=True)
print("Dropped Date column")

Dropped Date column


In [28]:
# Merging the training and testing dataset with the store dataset
train_df = pd.merge(train_df, store_df, on="Store")
test_df = pd.merge(test_df, store_df, on="Store")

In [29]:
train_df.head()

,Store,Sales,Open,Promo,StateHoliday,SchoolHoliday,DOW__1,DOW__2,DOW__3,DOW__4,...,Mean Customers Per Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5263,1,1,0,1,0,0,0,0,...,467.646497,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,5020,1,1,0,1,0,0,0,1,...,467.646497,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,4782,1,1,0,1,0,0,1,0,...,467.646497,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,5011,1,1,0,1,0,1,0,0,...,467.646497,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,6102,1,1,0,1,1,0,0,0,...,467.646497,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [30]:
# Time competition has been open for - in months
train_df['CompetitionOpen'] = 12 * (train_df["Year"] - train_df.CompetitionOpenSinceYear) + \
    (train_df["Month"] - train_df.CompetitionOpenSinceMonth)
train_df['CompetitionOpen'] = train_df.CompetitionOpen.apply(lambda x: x if x > 0 else 0)

In [31]:
test_df['CompetitionOpen'] = 12 * (test_df["Year"] - test_df.CompetitionOpenSinceYear) + \
    (test_df["Month"] - test_df.CompetitionOpenSinceMonth)
test_df['CompetitionOpen'] = test_df.CompetitionOpen.apply(lambda x: x if x > 0 else 0)

In [32]:
# Promo time in months
train_df['PromoOpen'] = 12 * (train_df.Year - train_df.Promo2SinceYear) + \
    (train_df.WeekofYear - train_df.Promo2SinceWeek) / float(4)
train_df['PromoOpen'] = train_df.PromoOpen.apply(lambda x: x if x > 0 else 0)
train_df.loc[train_df.Promo2SinceYear == 0, 'PromoOpen'] = 0

In [33]:
test_df['PromoOpen'] = 12 * (test_df.Year - test_df.Promo2SinceYear) + \
    (test_df.WeekofYear - test_df.Promo2SinceWeek) / float(4)
test_df['PromoOpen'] = test_df.PromoOpen.apply(lambda x: x if x > 0 else 0)
test_df.loc[test_df.Promo2SinceYear == 0, 'PromoOpen'] = 0

In [34]:
cols_to_drop = ["CompetitionOpenSinceMonth", "CompetitionOpenSinceYear", "Promo2SinceWeek", "Promo2SinceYear", 
               "PromoInterval"]

In [35]:
train_df.drop(cols_to_drop, axis=1, inplace=True)
test_df.drop(cols_to_drop, axis=1, inplace=True)

print("Dropped Columns")


Dropped Columns


### Preparing datasets

In [36]:
#Saving Id's for submission file
ID = test_df["Id"]
test_df.drop("Id", axis=1, inplace=True)

In [37]:
#print("Consider only open stores for training. Closed stores wont count into the score.")
#train_df = train_df[train_df["Open"] != 0]
print("Use only Sales bigger then zero. Simplifies calculation of rmspe")
train = train_df[train_df["Sales"] > 0]

Use only Sales bigger then zero. Simplifies calculation of rmspe


In [38]:
train_df.head()

,Store,Sales,Open,Promo,StateHoliday,SchoolHoliday,DOW__1,DOW__2,DOW__3,DOW__4,...,DayofYear,WeekofYear,Mean Sales Per Store,Mean Customers Per Store,StoreType,Assortment,CompetitionDistance,Promo2,CompetitionOpen,PromoOpen
0,1,5263,1,1,0,1,0,0,0,0,...,212,31,3945.704883,467.646497,3,1,1270.0,0,82.0,0.0
1,1,5020,1,1,0,1,0,0,0,1,...,211,31,3945.704883,467.646497,3,1,1270.0,0,82.0,0.0
2,1,4782,1,1,0,1,0,0,1,0,...,210,31,3945.704883,467.646497,3,1,1270.0,0,82.0,0.0
3,1,5011,1,1,0,1,0,1,0,0,...,209,31,3945.704883,467.646497,3,1,1270.0,0,82.0,0.0
4,1,6102,1,1,0,1,1,0,0,0,...,208,31,3945.704883,467.646497,3,1,1270.0,0,82.0,0.0


In [39]:
test_df["Open"] = test_df["Open"].astype(int)

In [40]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 26 columns):
Store                       1017209 non-null int64
Sales                       1017209 non-null int64
Open                        1017209 non-null int64
Promo                       1017209 non-null int64
StateHoliday                1017209 non-null int64
SchoolHoliday               1017209 non-null int64
DOW__1                      1017209 non-null uint8
DOW__2                      1017209 non-null uint8
DOW__3                      1017209 non-null uint8
DOW__4                      1017209 non-null uint8
DOW__5                      1017209 non-null uint8
DOW__6                      1017209 non-null uint8
DOW__7                      1017209 non-null uint8
Month                       1017209 non-null int64
Year                        1017209 non-null int64
Day                         1017209 non-null int64
DayofYear                   1017209 non-null int64
WeekofYear          

In [41]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41088 entries, 0 to 41087
Data columns (total 25 columns):
Store                       41088 non-null int64
Open                        41088 non-null int64
Promo                       41088 non-null int64
StateHoliday                41088 non-null int64
SchoolHoliday               41088 non-null int64
DOW__1                      41088 non-null uint8
DOW__2                      41088 non-null uint8
DOW__3                      41088 non-null uint8
DOW__4                      41088 non-null uint8
DOW__5                      41088 non-null uint8
DOW__6                      41088 non-null uint8
DOW__7                      41088 non-null uint8
Month                       41088 non-null int64
Year                        41088 non-null int64
Day                         41088 non-null int64
DayofYear                   41088 non-null int64
WeekofYear                  41088 non-null int64
Mean Sales Per Store        41088 non-null float64
Mean Cu

In [42]:
# For Keras
y_train = np.array(train_df["Sales"])
X_train = np.array(train_df.drop("Sales", axis=1))
test_df = np.array(test_df)

In [42]:
# For Xgboost
y_train = train_df["Sales"]
X_train = train_df.drop("Sales", axis=1)

## Models

#### Model 1 - XGB Regressor

In [43]:
import xgboost as xgb
model = xgb.XGBRegressor(n_estimators=700, max_depth=8,
                         nthread=-1, seed=7, colsample_bytree = 0.7,
                         subsample =  0.7, learning_rate=0.3)

/Users/pratheerthpadman/anaconda/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


**Public score of 0.16806**

#### Model 2 - Keras

In [43]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasRegressor

##define base model
def base_model():
     model = Sequential()
     model.add(Dense(400, input_dim=X_train.shape[1], init='normal'))
     model.add(Activation("relu"))
     model.add(Dropout(.1))
     model.add(Dense(128, init="normal"))
     model.add(Activation("relu"))                      
     model.add(Dense(1, init='normal'))
     model.compile(loss='mean_squared_error', optimizer = 'adamax')
     return model

seed = 7
np.random.seed(seed)

model = KerasRegressor(build_fn=base_model, epochs=20 , batch_size=128,verbose=1)

Using TensorFlow backend.


**Public score of 0.17660**

In [44]:
model.fit(X_train, y_train)

/Users/pratheerthpadman/anaconda/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, input_dim=25, kernel_initializer="normal")`
/Users/pratheerthpadman/anaconda/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_initializer="normal")`
/Users/pratheerthpadman/anaconda/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`


Epoch 1/10
1017209/1017209 [==============================] - 35s 35us/step - loss: 5985596.6565
Epoch 2/10
1017209/1017209 [==============================] - 35s 34us/step - loss: 2225187.6857
Epoch 3/10
1017209/1017209 [==============================] - 35s 34us/step - loss: 2043658.0769
Epoch 4/10
1017209/1017209 [==============================] - 35s 35us/step - loss: 1938267.1925
Epoch 5/10
1017209/1017209 [==============================] - 36s 35us/step - loss: 1877678.3433
Epoch 6/10
1017209/1017209 [==============================] - 36s 35us/step - loss: 1838887.2923
Epoch 7/10
1017209/1017209 [==============================] - 36s 36us/step - loss: 1799515.7075
Epoch 8/10
1017209/1017209 [==============================] - 37s 36us/step - loss: 1770405.5388
Epoch 9/10
1017209/1017209 [==============================] - 37s 36us/step - loss: 1736959.2333
Epoch 10/10
1017209/1017209 [==============================] - 44s 43us/step - loss: 1695821.7969


In [45]:
y_pred = model.predict(test_df)

41088/41088 [==============================] - 0s 11us/step


In [46]:
submission = pd.DataFrame()
submission['id'] = ID
submission['Sales'] = y_pred
submission.to_csv('Submission-keras15-sharper.csv', index=False)